In [1]:
import time
import streamlit as st 
from dotenv import load_dotenv
from processText import extractPdfText, textChunks, vectorDB
from chatWithPDF import get_conversation_chain
from templates import css, bot_template, user_template


import os
import shutil
from PyPDF2 import PdfReader
from langchain.schema import Document
from langchain.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

import chromadb
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [52]:
openai_api_key = ''
CHROMA_PATH = "chroma"
DATA_PATH = "data/"

In [3]:
def extractPdfText(documents):
    """
    Extracts raw text from the pdfs
    """
    text=""
    for pdf in documents:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text = text + page.extract_text()
            
    return text


def textChunks(text, save):
    """
    Splits the extracted text to chunks
    """
    if not save:
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
            )
        chunks = text_splitter.split_text(text)
        
    else:
        text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
        )
        chunks = text_splitter.create_documents([text])
        
    return chunks


def vectorDB(chunks):
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    #embeddings = openaiEmbeddings(OPEN_API_KEY=OPEN_API_KEY)
    vector_store = FAISS.from_texts(texts=chunks,
                                    embedding=embeddings)
    
    return vector_store


def chromaDB(chunks, name):
    # Clear out the database first.
    #if os.path.exists(CHROMA_PATH):
    #    shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    try:
        db = Chroma.from_documents(
        documents=documents, 
        embedding= OpenAIEmbeddings(openai_api_key=openai_api_key),
        persist_directory='db'
        )
        db.persist()
    return 'your' +  name + 'dabase has been created'
    except:
        return 'failed'








#########################################

def get_conversation_chain(db):
    llm = ChatOpenAI(openai_api_key=openai_api_key)
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm( 
    llm=llm,
    retriever = db.as_retriever(),
    memory = memory
    )

    return conversation_chain

<h2> 1 time - FAISS

In [4]:
from PyPDF2 import PdfReader

reader = PdfReader("data/Data_Scienstist.pdf")


In [5]:
page = reader.pages[0]
text = page.extract_text()

In [8]:
chunks = textChunks(text, False)

In [9]:
embeddingsDB = vectorDB(chunks)

In [10]:
model = get_conversation_chain(embeddingsDB)

In [11]:
model({'question': 'What is Spyros job'})

C:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'question': 'What is Spyros job',
 'chat_history': [HumanMessage(content='What is Spyros job'),
  AIMessage(content='Spyros is a Senior Data Scientist at Pfizer.')],
 'answer': 'Spyros is a Senior Data Scientist at Pfizer.'}

<h2> save load - ChromaDB

In [43]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [44]:
documents = textChunks(text, True)

In [45]:
print(documents[1])

page_content="2023\nTitle: Global Clinical Supplies\n \nAI alerts\n\uf002\nData Science Highlights\nData Science Highlights\nQuantum Computing /\n \nQuantum Machine Learning\n\uf085\nRelated interests\nRelated interests\nPfizer\nSenior Data Scientist\n | \nApr 2022\n - \nPresent\nEnd to end data science applications in Pfizers Global Clinical Supplies group.\n- Develop a NLP based application focusing on automating the reading,\n \nclassiﬁcation, and severity measurement of global events and emails received by\n \nPﬁzer's Global Security Centers. The application aimed to identify potential risks\n \nfor the company's clinical centers.\n- Created and maintained dashboards and web applications to sustain predictive\n \nmodels pertaining to global clinical sites. Speciﬁcally, focused on packing and\n \nlabeling processes and inventory lot monitoring.\n- Supervised college interns within Pﬁzer's organization who were in the process\n \nof completing their dissertations. Provided guidance a

In [46]:
chromadb = Chroma.from_documents(documents=documents, 
                                 embedding= OpenAIEmbeddings(openai_api_key=openai_api_key),
                                 persist_directory='bla')

In [47]:
chromadb.persist()

In [48]:
vectordb = Chroma(persist_directory='bla', 
                  embedding_function=OpenAIEmbeddings(openai_api_key=openai_api_key))

In [49]:
# https://www.youtube.com/watch?v=cFCGUjc33aU

In [50]:
retriever = vectordb.as_retriever()

In [51]:
docs = retriever.get_relevant_documents("What is Spyros job?")

In [38]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000131998B9750>)

In [41]:
def get_conversation_chain(db):
    llm = ChatOpenAI(openai_api_key=openai_api_key)
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm( 
    llm=llm,
    retriever = db,
    memory = memory
    )

    return conversation_chain

In [42]:
get_conversation_chain(retriever)

ConversationalRetrievalChain(memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history'), combine_docs_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000131992A8C10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000013199984750>, openai_api_key=SecretStr('**********'), openai_proxy='')), document_variable_name='context'), question_generator=LLMChain(prompt=PromptTemplate(input_variables=['chat_hi

In [103]:
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=openai_api_key), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [104]:
llm_response = qa_chain("What is Spyros job?")

In [106]:
llm_response

{'query': 'What is Spyros job?',
 'result': ' Spyros is a Data Scientist.',
 'source_documents': [Document(page_content='Spyros\nGeorgopoulos\nData Scientist\nContact\n14, rue Chingiz T. Aitmatov\nL-1161 Luxembourg ville\n\uf015\n+30 6947508036\n\uf10b\nspyros.p.georgopoulos@gmail.com\n\uf0e0\nSocial\nhttps://www.linkedin.com/in/georgop\noulos-spyridon\n\uf0e1\nhttps://github.com/ownedbyphysics?\ntab=repositories\n\uf09b\nPandas, NumPy, Matplotlib,\n \nSeaborn, Plotly, Scikit-learn,\n \nNLTK, SpaCy, Keras,\n \nTensorFlow, PyTorch,\n \nTransformers, Langchain,\n \nOpenAI, SciPy, Scrapy,\n  \nBeautifulSoup, OpenCV\n\uf00c\nHTML - CSS - XPath\n\uf00c\nFlask, Dash, Spotﬁre,\n \nTableau, Git, Unit Testing,\n \nTerraform scripts\n\uf00c\nAWS, DSS (Dataiku)\n\uf00c\nSkills\nSkills\nMain author in a peer-\nreviewed publication\n \n(Inderscience\n \nsumbissions)\nTitle: Reservoir Computing vs.\n \nNeural Networks in Financial\n \nForecasting\n\uf002\nPoster presentation in the\n \nAnalytics Sum